# EDA Lab + Homework (Instructor)

Solutions + short rationales. Keep answers concise and interview-oriented.


In [ ]:
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f"Failed: {name}")
    print(f"OK: {name}")

rng = np.random.default_rng(0)

CSV = """user_id,country,plan,signup_days_ago,tenure_days,sessions_last_7d,avg_session_min,age,discount_pct,last_payment_failed,churned,refund_after_churn_flag
1,US,basic,400,365,2,8.0,25,0.0,0,0,0
2,US,premium,120,110,18,22.0,41,10.0,0,0,0
3,IN,basic,30,25,1,3.0,19,0.0,1,1,1
4,IN,basic,10,7,0,2.0,22,0.0,1,0,0
5,GB,premium,800,790,30,35.0,52,15.0,0,0,0
6,GB,basic,200,180,5,10.0,33,,1,1,1
7,US,basic,60,55,7,12.0,,0.0,0,0,0
8,CA,premium,90,80,14,18.0,29,5.0,0,0,0
9,CA,basic,15,12,0,1.0,24,0.0,1,1,1
10,IN,premium,300,280,20,25.0,38,20.0,0,0,0
11,US,premium,5,3,0,0.5,27,0.0,1,1,1
12,GB,premium,45,40,9,15.0,31,0.0,0,0,0
13,US,basic,500,490,1,6.0,47,0.0,0,0,0
14,IN,basic,75,70,4,9.0,28,0.0,0,0,0
15,CA,basic,365,360,3,7.0,36,0.0,0,0,0
16,US,premium,20,18,0,1.0,23,0.0,1,1,1
17,GB,basic,150,140,6,11.0,42,0.0,0,0,0
18,IN,premium,220,210,25,28.0,34,25.0,0,0,0
19,US,basic,12,10,0,0.2,21,0.0,1,1,1
20,CA,premium,600,590,22,26.0,55,0.0,0,0,0
"""

df = pd.read_csv(io.StringIO(CSV))
check('n_rows', df.shape[0] == 20)
df.head()


## Solutions


### 1.1 Grain + schema


Grain: **one row per user** (user-level snapshot).


In [ ]:
df.dtypes
# Note: missing numeric values become NaN; that's expected for EDA.


### 1.2 Missingness + duplicates


In [ ]:
def missingness_table(df: pd.DataFrame) -> pd.DataFrame:
    n = len(df)
    n_missing = df.isna().sum()
    pct = (n_missing / n).sort_values(ascending=False)
    out = pd.DataFrame({"n_missing": n_missing, "pct_missing": (n_missing / n)})
    return out.sort_values("pct_missing", ascending=False)

miss = missingness_table(df)
miss
check('miss_has_cols', set(miss.columns) >= {'n_missing','pct_missing'})

n_dup_rows = int(df.duplicated().sum())
n_dup_user_id = int(df["user_id"].duplicated().sum())
print(n_dup_rows, n_dup_user_id)


### 2.1 Numeric summaries


In [ ]:
num = df.select_dtypes(include=[np.number])
summary = num.describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95]).T
summary


### 2.2 High-signal plots


In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.hist(df["sessions_last_7d"], bins=10)
plt.title("sessions_last_7d (skew)")
plt.subplot(1,2,2)
plt.hist(df["avg_session_min"], bins=10)
plt.title("avg_session_min (heavy tail)")
plt.tight_layout()
plt.show()


### 3.1 Label imbalance


In [ ]:
churn_rate = float(df["churned"].mean())
counts = df["churned"].value_counts().rename_axis("churned").to_frame("n")
print("churn_rate:", churn_rate)
counts
# Rationale: if churn is rare, accuracy is misleading; prefer PR-AUC / F1 / cost-sensitive metrics.


### 3.2 Slice analysis


In [ ]:
by_country = df.groupby("country")["churned"].mean().sort_values(ascending=False)
by_plan = df.groupby("plan")["churned"].mean().sort_values(ascending=False)
by_country, by_plan

df2 = df.copy()
df2["tenure_bucket"] = pd.cut(df2["tenure_days"], bins=[-1, 14, 60, 180, 10_000], labels=["<=2w","2w-2m","2m-6m",">6m"])
by_bucket = df2.groupby("tenure_bucket")["churned"].mean().sort_values(ascending=False)
by_bucket


### 4.1 Leakage-prone features


In [ ]:
leaky_cols = ["refund_after_churn_flag"]
leaky_cols
# Rationale: this column is defined after churn outcome; it would not exist at prediction time.
# It can trivially predict churn (near-perfect correlation), inflating offline metrics.


## Example Homework Write-up (brief)


- Missingness: `age` and `discount_pct` have NaNs; decide imputation policy.
- Potential data quality: some users have 0 sessions but high tenure; verify instrumentation.
- Imbalance: churn rate should be tracked; avoid accuracy as sole metric.
- Slices: churn differs by plan/country/tenure bucket; consider stratified evaluation.
- Leakage: `refund_after_churn_flag` must be excluded from training features.
